<h1><center>Inteligencia Artificial</center></h1>


<h2><center>Tarea 2 - Aspiradora Inteligente</center></h2>

---

#### El **objetivo** de esta tarea será implementar algoritmos de búsqueda para que la aspiradora pueda "ser inteligente".

* Con base a lo que se ha visto del perro robot y lo respectivo de los agentes, ¿será posible realizar el mismo ejercicio para que una aspiradora en un espacio de 2D (6x6) y pueda concluir de limpiar? Los resultados se deben imprimir en 'consola'.

* Además, deberán agregar su informacción correspondiente del equipo que hayan formado, así como el link de github correspondiente al cual cargarán la notebook (ejecutada previamente), para visualizar las respuestas.

---
Edna Cruz Flores, 10034119, Maestría en Optimización y Cómputo Aplicado, 3er semestre.

Alejandro Carvantes Barrera
10036987
MAESTRIA EN OPTIMIZACION Y COMPUTO APLICADO
1


Ollin Villalon Villarreal
10032484
Licenciatura en ciencias computacionales

---

### Modifiquen las clases que se han definido hasta este momento del Parque en 2D y el comportamiento del perro:

* cuarto = Cuarto2D(6,6, color={'Aqui la aspiradora': (50,50,50), 'Basura': (200, 200, 200)}) (por ejemplo)

* Actualización 1: añadir obstáculos y que la aspiradora determine aleatoriamente a donde moverse para continuar.

* Actualización 2: consistirá en la implementación de un algoritmo de búsqueda para determinar cuál sería un camino para limpiar la habitación (todavía sin obstáculos), expandiendo a una habitación de 10x10.

* Actualización 3: añadir obstáculos a dicha habitación para que realice su proceso de limpieza.

In [1]:
from agents import *
from notebook import psource
import numpy as np

In [2]:
class Thing:
    """This represents any physical object that can appear in an Environment.
    You subclass Thing to get the things you want. Each thing can have a
    .__name__  slot (used for output only)."""

    def __repr__(self):
        return '<{}>'.format(getattr(self, '__name__', self.__class__.__name__))

    def is_alive(self):
        """Things that are 'alive' should return true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Display the agent's internal state. Subclasses should override."""
        print("I don't know how to show_state.")

    def display(self, canvas, x, y, width, height):
        """Display an image of this Thing on the canvas."""
        # Do we need this?
        pass


class Agent(Thing):
    """An Agent is a subclass of Thing with one required instance attribute 
    (aka slot), .program, which should hold a function that takes one argument,
    the percept, and returns an action. (What counts as a percept or action 
    will depend on the specific environment in which the agent exists.)
    Note that 'program' is a slot, not a method. If it were a method, then the
    program could 'cheat' and look at aspects of the agent. It's not supposed
    to do that: the program can only look at the percepts. An agent program
    that needs a model of the world (and of the agent itself) will have to
    build and maintain its own model. There is an optional slot, .performance,
    which is a number giving the performance measure of the agent in its
    environment."""

    def __init__(self, program=None):
        self.alive = True
        self.bump = False
        self.holding = []
        self.performance = 0
        if program is None or not isinstance(program, collections.abc.Callable):
            print("Can't find a valid program for {}, falling back to default.".format(self.__class__.__name__))

            def program(percept):
                return eval(input('Percept={}; action? '.format(percept)))

        self.program = program

    def can_grab(self, thing):
        """Return True if this agent can grab this thing.
        Override for appropriate subclasses of Agent and Thing."""
        return False

In [3]:
class Environment:
    """Abstract class representing an Environment. 'Real' Environment classes
    inherit from this. Your Environment will typically need to implement:
        percept:           Define the percept that an agent sees.
        execute_action:    Define the effects of executing an action.
                           Also update the agent.performance slot.
    The environment keeps a list of .things and .agents (which is a subset
    of .things). Each agent has a .performance slot, initialized to 0.
    Each thing has a .location slot, even though some environments may not
    need this."""

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Return the percept that the agent sees at this point. (Implement this.)"""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """Change the world to reflect this action. (Implement this.)"""
        raise NotImplementedError

    def default_location(self, thing):
        """Default location to place a new thing with unspecified location."""
        return None

    def exogenous_change(self):
        """If there is spontaneous change in the world, override this."""
        pass

    def is_done(self):
        """By default, we're done when we can't find a live agent."""
        return not any(agent.is_alive() for agent in self.agents)

    def step(self):
        """Run the environment for one time step. If the
        actions and exogenous changes are independent, this method will
        do. If there are interactions between them, you'll need to
        override this method."""
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)
            self.exogenous_change()

    def run(self, steps=1000):
        """Run the Environment for given number of time steps."""
        for step in range(steps):
            if self.is_done():
                return
            self.step()

    def list_things_at(self, location, tclass=Thing):
        """Return all things exactly at a given location."""
        if isinstance(location, numbers.Number):
            return [thing for thing in self.things
                    if thing.location == location and isinstance(thing, tclass)]
        return [thing for thing in self.things
                if all(x == y for x, y in zip(thing.location, location)) and isinstance(thing, tclass)]

    def some_things_at(self, location, tclass=Thing):
        """Return true if at least one of the things at location
        is an instance of class tclass (or a subclass)."""
        return self.list_things_at(location, tclass) != []

    def add_thing(self, thing, location=None):
        """Add a thing to the environment, setting its location. For
        convenience, if thing is an agent program we make a new agent
        for it. (Shouldn't need to override this.)"""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        if thing in self.things:
            print("Can't add the same thing twice")
        else:
            thing.location = location if location is not None else self.default_location(thing)
            self.things.append(thing)
            if isinstance(thing, Agent):
                thing.performance = 0
                self.agents.append(thing)

    def delete_thing(self, thing):
        """Remove a thing from the environment."""
        try:
            self.things.remove(thing)
        except ValueError as e:
            print(e)
            print("  in Environment delete_thing")
            print("  Thing to be removed: {} at {}".format(thing, thing.location))
            print("  from list: {}".format([(thing, thing.location) for thing in self.things]))
        if thing in self.agents:
            self.agents.remove(thing)

In [4]:
class Direction:
    """A direction class for agents that want to move in a 2D plane
        Usage:
            d = Direction("down")
            To change directions:
            d = d + "right" or d = d + Direction.R #Both do the same thing
            Note that the argument to __add__ must be a string and not a Direction object.
            Also, it (the argument) can only be right or left."""

    R = "right"
    L = "left"
    U = "up"
    D = "down"

    def __init__(self, direction):
        self.direction = direction

    def __add__(self, heading):
        """
        >>> d = Direction('right')
        >>> l1 = d.__add__(Direction.L)
        >>> l2 = d.__add__(Direction.R)
        >>> l1.direction
        'up'
        >>> l2.direction
        'down'
        >>> d = Direction('down')
        >>> l1 = d.__add__('right')
        >>> l2 = d.__add__('left')
        >>> l1.direction == Direction.L
        True
        >>> l2.direction == Direction.R
        True
        """
        if self.direction == self.R:
            return {
                self.R: Direction(self.D),
                self.L: Direction(self.U),
            }.get(heading, None)
        elif self.direction == self.L:
            return {
                self.R: Direction(self.U),
                self.L: Direction(self.D),
            }.get(heading, None)
        elif self.direction == self.U:
            return {
                self.R: Direction(self.R),
                self.L: Direction(self.L),
            }.get(heading, None)
        elif self.direction == self.D:
            return {
                self.R: Direction(self.L),
                self.L: Direction(self.R),
            }.get(heading, None)

    def move_forward(self, from_location):
        """
        >>> d = Direction('up')
        >>> l1 = d.move_forward((0, 0))
        >>> l1
        (0, -1)
        >>> d = Direction(Direction.R)
        >>> l1 = d.move_forward((0, 0))
        >>> l1
        (1, 0)
        """
        # get the iterable class to return
        iclass = from_location.__class__
        x, y = from_location
        if self.direction == self.R:
            return iclass((x + 1, y))
        elif self.direction == self.L:
            return iclass((x - 1, y))
        elif self.direction == self.U:
            return iclass((x, y - 1))
        elif self.direction == self.D:
            return iclass((x, y + 1))


class XYEnvironment(Environment):
    """This class is for environments on a 2D plane, with locations
    labelled by (x, y) points, either discrete or continuous.

    Agents perceive things within a radius. Each agent in the
    environment has a .location slot which should be a location such
    as (0, 1), and a .holding slot, which should be a list of things
    that are held."""

    def __init__(self, width=10, height=10):
        super().__init__()

        self.width = width
        self.height = height
        self.observers = []
        # Sets iteration start and end (no walls).
        self.x_start, self.y_start = (0, 0)
        self.x_end, self.y_end = (self.width-1, self.height-1)

    perceptible_distance = 1

    def things_near(self, location, radius=None):
        """Return all things within radius of location."""
        if radius is None:
            radius = self.perceptible_distance
        radius2 = radius * radius
        return [(thing, radius2 - distance_squared(location, thing.location))
                for thing in self.things if distance_squared(
                location, thing.location) <= radius2]

    def percept(self, agent):
        """By default, agent perceives things within a default radius."""
        return self.things_near(agent.location)

    def execute_action(self, agent, action):
        agent.bump = False
        if action == 'TurnRight':
            agent.direction += Direction.R
        elif action == 'TurnLeft':
            agent.direction += Direction.L
        elif action == 'Forward':
            agent.bump = self.move_to(agent, agent.direction.move_forward(agent.location))
        elif action == 'Grab':
            things = [thing for thing in self.list_things_at(agent.location) if agent.can_grab(thing)]
            if things:    
                agent.holding.append(things[0])
                print("Grabbing ", things[0].__class__.__name__)
                self.delete_thing(things[0])
        elif action == 'Release':
            if agent.holding:
                dropped = agent.holding.pop()
                print("Dropping ", dropped.__class__.__name__)
                self.add_thing(dropped, location=agent.location)

    def default_location(self, thing):
        location = self.random_location_inbounds()
        while self.some_things_at(location, Obstacle):
            # we will find a random location with no obstacles
            location = self.random_location_inbounds()
        return location

    def move_to(self, thing, destination):
        """Move a thing to a new location. Returns True on success or False if there is an Obstacle.
        If thing is holding anything, they move with him."""
        thing.bump = self.some_things_at(destination, Obstacle)
        if not thing.bump:
            thing.location = destination
            for o in self.observers:
                o.thing_moved(thing)
            for t in thing.holding:
                self.delete_thing(t)
                self.add_thing(t, destination)
                t.location = destination
        return thing.bump

    def add_thing(self, thing, location=None, exclude_duplicate_class_items=False):
        """Add things to the world. If (exclude_duplicate_class_items) then the item won't be
        added if the location has at least one item of the same class."""
        if location is None:
            super().add_thing(thing)
        elif self.is_inbounds(location):
            if (exclude_duplicate_class_items and
                    any(isinstance(t, thing.__class__) for t in self.list_things_at(location))):
                return
            super().add_thing(thing, location)

    def is_inbounds(self, location):
        """Checks to make sure that the location is inbounds (within walls if we have walls)"""
        x, y = location
        return not (x < self.x_start or x > self.x_end or y < self.y_start or y > self.y_end)

    def random_location_inbounds(self, exclude=None):
        """Returns a random location that is inbounds (within walls if we have walls)"""
        location = (random.randint(self.x_start, self.x_end),
                    random.randint(self.y_start, self.y_end))
        if exclude is not None:
            while location == exclude:
                location = (random.randint(self.x_start, self.x_end),
                            random.randint(self.y_start, self.y_end))
        return location

    def delete_thing(self, thing):
        """Deletes thing, and everything it is holding (if thing is an agent)"""
        if isinstance(thing, Agent):
            del thing.holding

        super().delete_thing(thing)
        for obs in self.observers:
            obs.thing_deleted(thing)

    def add_walls(self):
        """Put walls around the entire perimeter of the grid."""
        for x in range(self.width):
            self.add_thing(Wall(), (x, 0))
            self.add_thing(Wall(), (x, self.height - 1))
        for y in range(1, self.height - 1):
            self.add_thing(Wall(), (0, y))
            self.add_thing(Wall(), (self.width - 1, y))

        # Updates iteration start and end (with walls).
        self.x_start, self.y_start = (1, 1)
        self.x_end, self.y_end = (self.width - 1, self.height - 1)

    def add_observer(self, observer):
        """Adds an observer to the list of observers.
        An observer is typically an EnvGUI.

        Each observer is notified of changes in move_to and add_thing,
        by calling the observer's methods thing_moved(thing)
        and thing_added(thing, loc)."""
        self.observers.append(observer)

    def turn_heading(self, heading, inc):
        """Return the heading to the left (inc=+1) or right (inc=-1) of heading."""
        return turn_heading(heading, inc)


class Obstacle(Thing):
    """Something that can cause a bump, preventing an agent from
    moving into the same square it's in."""
    pass


class Wall(Obstacle):
    pass


# ______________________________________________________________________________


class GraphicEnvironment(XYEnvironment):
    def __init__(self, width=10, height=10, boundary=True, color={}, display=False):
        """Define all the usual XYEnvironment characteristics,
        but initialise a BlockGrid for GUI too."""
        super().__init__(width, height)
        self.grid = BlockGrid(width, height, fill=(200, 200, 200))
        if display:
            self.grid.show()
            self.visible = True
        else:
            self.visible = False
        self.bounded = boundary
        self.colors = color

    def get_world(self):
        """Returns all the items in the world in a format
        understandable by the ipythonblocks BlockGrid."""
        result = []
        x_start, y_start = (0, 0)
        x_end, y_end = self.width, self.height
        for x in range(x_start, x_end):
            row = []
            for y in range(y_start, y_end):
                row.append(self.list_things_at((x, y)))
            result.append(row)
        return result

    """
    def run(self, steps=1000, delay=1):
        "" "Run the Environment for given number of time steps,
        but update the GUI too." ""
        for step in range(steps):
            sleep(delay)
            if self.visible:
                self.reveal()
            if self.is_done():
                if self.visible:
                    self.reveal()
                return
            self.step()
        if self.visible:
            self.reveal()
    """

    def run(self, steps=1000, delay=1):
        """Run the Environment for given number of time steps,
        but update the GUI too."""
        for step in range(steps):
            self.update(delay)
            if self.is_done():
                break
            self.step()
        self.update(delay)

    def update(self, delay=1):
        sleep(delay)
        self.reveal()

    def reveal(self):
        """Display the BlockGrid for this world - the last thing to be added
        at a location defines the location color."""
        self.draw_world()
        # wait for the world to update and
        # apply changes to the same grid instead
        # of making a new one.
        clear_output(1)
        self.grid.show()
        self.visible = True

    def draw_world(self):
        self.grid[:] = (200, 200, 200)
        world = self.get_world()
        for x in range(0, len(world)):
            for y in range(0, len(world[x])):
                if len(world[x][y]):
                    self.grid[y, x] = self.colors[world[x][y][-1].__class__.__name__]

    def conceal(self):
        """Hide the BlockGrid for this world"""
        self.visible = False
        display(HTML(''))

---

# Aquí Actualización 1

---

In [5]:
# ACTUALIZACION 1
from random import choice

class Bump(Thing):
    pass

class Trash(Thing):
    pass

class Wall(Obstacle):
    pass

class EnergeticRumbaCleaner(Agent):
    def __init__(self, program, loc=[0,0]):
        super().__init__(program)
        self.direction = Direction(choice(("up", "down", "left", "right")))
        self.location = loc

def program(percepts):
    for cosa in percepts:
        if isinstance(cosa, Trash):
            return "Aspira"
        elif choice(("move", "turn")) == "move":
            return "Forward"
        else:
            return choice(("TurnRight", "TurnLeft"))

class Room2D(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)[::-1]
        print(agent.location)
        return things
    
    def execute_action(self, agent, action):
        agent.bump = False
        if action == 'TurnRight':
            agent.direction += Direction.R
        elif action == 'TurnLeft':
            agent.direction += Direction.L
        elif action == 'Forward' and self.is_inbounds(agent.direction.move_forward(agent.location)):
            agent.bump = self.move_to(agent, agent.direction.move_forward(agent.location))
        elif action == 'Aspira':
            things = [thing for thing in self.list_things_at(agent.location) if isinstance(thing, Trash)]
            if things:
                print("Aspirando ", things[0].__class__.__name__)
                self.delete_thing(things[0])
                    
    def is_done(self):
        no_trash = not any(isinstance(thing, Trash) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_trash


In [6]:
# Actualizacion 1
from ipythonblocks import BlockGrid
import random
import copy
import collections
import numbers
from time import sleep
from IPython.display import HTML, display, clear_output

#Aquí su código

room = Room2D(10, 10, True, {"Wall":(0, 0, 0), "Trash": (150, 100, 100), "EnergeticRumbaCleaner":(255, 0, 0)}, True)
aspiradora = EnergeticRumbaCleaner(program, [5,5])#[1, 1])
room.add_thing(aspiradora, aspiradora.location)
room.add_thing(Trash(), [5, 4])
room.add_thing(Trash(), [5, 6])
room.add_thing(Trash(), [6, 5])
room.add_thing(Trash(), [4, 5])
room.add_thing(Trash(), [0, 0])

# room.add_walls()
# print(isinstance(Trash(), Trash))
room.run(30)
# grid = BlockGrid(5,5)
# print(grid[4,3])
# grid[2] = (155, 100, 100)
# grid.show()
# psource(BlockGrid)

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


[5, 5]


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


[4, 5]
Aspirando  Trash


...

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


[0, 0]


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


[0, 0]


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


---

# Aquí Actualización 2

---

In [7]:
class Queue:
    queue = []
    def __init__(self):
        pass
        

In [8]:
# ACTUALIZACION 2
from random import choice

class Trash(Thing):
    pass

class Wall(Obstacle):
    pass

class Vertex:
    # se inicia el vértice con las coordenadas x, y
    def __init__(self,x_coord,y_coord, things):
        self.x=x_coord
        self.y=y_coord
        # se inicializa la distancia como infinita para que cualqueir distancia sea menor que el valor inicial
        self.d=float('inf') #distance from source
        self.parent_x=None
        self.parent_y=None
        self.processed=True if any(isinstance(t, Wall) for t in things) else False
        self.index_in_queue=None




def get_neighbors(mat,r,c):
#     shape=mat.shape
    shape = [len(mat), len(mat[0])]
    neighbors=[]
    if r > 0 and not mat[r-1][c].processed:
        neighbors.append(mat[r-1][c])
    if r < shape[0] - 1 and not mat[r+1][c].processed:
        neighbors.append(mat[r+1][c])
    if c > 0 and not mat[r][c-1].processed:
        neighbors.append(mat[r][c-1])
    if c < shape[1] - 1 and not mat[r][c+1].processed:
        neighbors.append(mat[r][c+1])
    return neighbors

def bubble_up(queue, index):
    if index <= 0:
        return queue
    p_index=(index-1)//2
    if queue[index].d < queue[p_index].d:
        queue[index], queue[p_index]=queue[p_index], queue[index]
        queue[index].index_in_queue=index
        queue[p_index].index_in_queue=p_index
        queque = bubble_up(queue, p_index)
    return queue
    
def bubble_down(queue, index):
    length=len(queue)
    lc_index=2*index+1
    rc_index=lc_index+1
    if lc_index >= length:
        return queue
    if rc_index == length:
        if queue[index].d > queue[lc_index].d:
            queue[index], queue[lc_index]=queue[lc_index], queue[index]
            queue[index].index_in_queue=index
            queue[lc_index].index_in_queue=lc_index
            queue = bubble_down(queue, lc_index)
    else:
        small = lc_index
        if queue[lc_index].d > queue[rc_index].d:
            small = rc_index
        if queue[small].d < queue[index].d:
            queue[index],queue[small]=queue[small],queue[index]
            queue[index].index_in_queue=index
            queue[small].index_in_queue=small
            queue = bubble_down(queue, small)
    return queue

def get_distance(u,v):
    return 0.01 + abs(v[0] - u[0]) + abs(v[1] - u[1])

class EnergeticRumbaCleaner(Agent):
    def __init__(self, program, loc=[0,0]):
        super().__init__(program)
        self.direction = Direction(choice(("up", "down", "left", "right")))
        self.location = loc

def program(percepts):
    for cosa in percepts:
        if isinstance(cosa, Trash):
            return "Aspira"
        else:
            return "Busca"

class Room2D(GraphicEnvironment):
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''

        things = self.list_things_at(agent.location)[::-1]
        
        return things
    
    def calculatePath(self, agent):
        if not self.things:
            return

        minDist = float('inf')
        
        for cosa in self.things:
            if isinstance(cosa, Trash) and get_distance(agent.location, cosa.location)<minDist:
                minDist = get_distance(agent.location, cosa.location)
                dst = cosa.location
                
        src = agent.location

        pq=[]

        matrix = []
        
        for x in range(self.width):
            matrix.append([])
            for y in range(self.height):
            
                matrix[x].append(Vertex(x, y, self.list_things_at([x, y])))
                matrix[x][y].index_in_queue=len(pq)
                pq.append(matrix[x][y])

        # guarda la coordenada x de la posición inicial
        source_x=src[0]
        # guarda la coordenada y de la posición inicial
        source_y=src[1]
        # guarda la coordenada x de la posición final
        dest_x=dst[0]
        # guarda la coordenada y de la posición final
        dest_y=dst[1]
        
        matrix[source_x][source_y].d=0
        # movemos el vertice de inicio en la priority queue
        pq=bubble_up(pq, matrix[source_x][source_y].index_in_queue)
        # mientras queden vértices en la queue
        while len(pq) > 0:
            # leemos el primer elemento, que es el vertice con mayor prioridad en la queue
            u=pq[0]
            # lo etiquetamos como procesado
            u.processed=True
            # movemos el último vertice de la queue al inicio para remover a u de la queue
            pq[0]=pq[-1]
            # reajustamos el indice que tiene guardado
            pq[0].index_in_queue=0 
            # removemos del queue el último elemento
            pq.pop()
            # reacomodamos la heap, moviendo hacia abajo el vértice que movimos al inicio
            pq=bubble_down(pq,0)
            # ya que tenemos reacomodada la queue
            # obtenemos los vecinos de u, el vértice que acabamos de quitar te la queue
            neighbors = get_neighbors(matrix,u.x,u.y) 
            # para cada uno de los vecinos
            for v in neighbors:
                # calculamos la distancia
                dist=get_distance((u.x,u.y),(v.x,v.y)) 
                # si la distancia recorrida hasta el vertice u más la distancia de de u a v es menor que la distancia registrada para llegar a v 
                if u.d + dist < v.d: 
                    # entonces encontramos un camino más corto hasta v y reajustamos la distancia
                    v.d = u.d+dist
                    # registamos la coordenada x del vertice por el cual encontramos un camino más corto
                    v.parent_x=u.x
                    # registamos la coordenada y del vertice por el cual encontramos un camino más corto
                    v.parent_y=u.y
                    # registramos la posición que tiene v en la queue
                    idx=v.index_in_queue
                    # reacomodamos el vertice v en la queue de acuerdo a su nueva distancia
                    pq=bubble_down(pq,idx)
                    pq=bubble_up(pq,idx)
        # iniciamos un arreglo vacío para guardar el camino
        path=[]
        # agregaremos los vertices del camino iniciando por el final
        iter_v=matrix[dest_x][dest_y]

        # como iniciamos por el final, seguimos hasta que llegemos al principio
        while(iter_v.y!=source_y or iter_v.x!=source_x):
            # agregamos el vértice al inicio del array path
            path.insert(0,(iter_v.x,iter_v.y))
            # y guardamos el vértice padre del vértice que acabamos de agregar al camino
            iter_v=matrix[iter_v.parent_x][iter_v.parent_y]
        # una vez que llegamos al vértice inicial, lo agregamos al camino
        path.insert(0, (source_x,source_y))
        self.path_to_trash = path
        return path

    def execute_action(self, agent, action):
        agent.bump = False
        if action == 'TurnRight':
            agent.direction += Direction.R
        elif action == 'TurnLeft':
            agent.direction += Direction.L
        elif action == 'Forward' and self.is_inbounds(agent.direction.move_forward(agent.location)):
            agent.bump = self.move_to(agent, agent.direction.move_forward(agent.location))
        elif action == 'Aspira':
            things = [thing for thing in self.list_things_at(agent.location) if isinstance(thing, Trash)]
            if things:
                print("Aspirando ", things[0].__class__.__name__)
                self.delete_thing(things[0])
        elif action == "Busca":
            if self.path_to_trash:
                self.move_to(agent, self.path_to_trash.pop(0))
            else:
                self.calculatePath(agent)

    def is_done(self):
        no_trash = not any(isinstance(thing, Trash) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_trash

In [9]:
# Actualizacion 2
#Aquí su código
class Wall(Obstacle):
    pass

room = Room2D(14, 17, True, {"Wall":(0, 0, 0), "Trash": (150, 100, 100), "EnergeticRumbaCleaner":(255, 0, 0)}, True)
RumbaCleaner = EnergeticRumbaCleaner(program)
room.add_thing(RumbaCleaner, [0,0])
room.add_thing(Trash(), [1, 2])
room.add_thing(Trash(), [2, 4])
room.add_thing(Trash(), [7, 7])

# room.add_thing(Wall(), [0, 2])
# room.add_thing(Wall(), [1, 2])
# room.add_thing(Wall(), [2, 2])
# room.add_thing(Wall(), [3, 2])
# room.add_thing(Wall(), [4, 2])

room.calculatePath(RumbaCleaner)
room.run(50)


,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,


,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,


,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,


,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,


,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,


...

,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,


,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,


Aspirando  Trash


,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,


,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,


---

# Aquí Actualización 3

---

In [10]:
# Actualizacion 3
#Aquí su código

class Wall(Obstacle):
    pass

room = Room2D(10, 10, True, {"Wall":(0, 0, 0), "Trash": (150, 100, 100), "EnergeticRumbaCleaner":(255, 0, 0)}, True)
RumbaCleaner = EnergeticRumbaCleaner(program)

room.add_thing(RumbaCleaner, [0,0])
room.add_thing(Trash(), [2, 4])
room.add_thing(Trash(), [7, 7])
room.add_thing(Trash(), [9, 9])




room.add_thing(Wall(), [0, 1])
room.add_thing(Wall(), [1, 1])
room.add_thing(Wall(), [2, 1])
room.add_thing(Wall(), [3, 1])
room.add_thing(Wall(), [4, 1])
room.add_thing(Wall(), [5, 1])
room.add_thing(Wall(), [6, 1])
room.add_thing(Wall(), [7, 1])
room.add_thing(Wall(), [8, 1])
room.add_thing(Wall(), [8, 2])
room.add_thing(Wall(), [8, 3])
room.add_thing(Wall(), [8, 4])
room.add_thing(Wall(), [8, 5])
room.add_thing(Wall(), [8, 6])
room.add_thing(Wall(), [8, 7])
room.add_thing(Wall(), [8, 8])
room.add_thing(Wall(), [7, 8])
room.add_thing(Wall(), [6, 8])
room.add_thing(Wall(), [5, 8])
room.add_thing(Wall(), [4, 8])
room.add_thing(Wall(), [3, 8])
room.add_thing(Wall(), [2, 8])
room.add_thing(Wall(), [1, 8])
room.add_thing(Wall(), [1, 7])
room.add_thing(Wall(), [1, 6])
room.add_thing(Wall(), [1, 5])
room.add_thing(Wall(), [1, 4])
room.add_thing(Wall(), [1, 3])
room.add_thing(Wall(), [2, 3])
room.add_thing(Wall(), [3, 3])
room.add_thing(Wall(), [4, 3])
room.add_thing(Wall(), [5, 3])
room.add_thing(Wall(), [6, 3])

room.calculatePath(RumbaCleaner)
room.run(100)

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


...

,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


Aspirando  Trash


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,


,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
,,,,,,,,,
